<a href="https://colab.research.google.com/github/tylerle29/Memotron/blob/main/Copy_of_NVIDIA_Nemotron_Nano_9B_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U transformers

trust_remote_code=True

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 11.6 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 11.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.33.1
    Uninstalling huggingface-hub-0.33.1:
      Successfully uninstalled huggingface-hub-0.33.1
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.0
    Uninstalling transformers-4.53.0:
      Successfully uninstalled transformers-4.53.0


In [6]:
!pip install -U mamba-ssm

  Using cached mamba_ssm-2.2.6.post3.tar.gz (113 kB)
  Installing build dependencies ... /^C
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/memeatron/bin/pip", line 11, in <module>
    sys.exit(main())
             ~~~~^^
  File "/opt/homebrew/Caskroom/miniconda/base/envs/memeatron/lib/python3.13/site-packages/pip/_internal/cli/main.py", line 80, in main
    return command.main(cmd_args)
           ~~~~~~~~~~~~^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/envs/memeatron/lib/python3.13/site-packages/pip/_internal/cli/base_command.py", line 157, in main
    with self.main_context():
         ~~~~~~~~~~~~~~~~~^^
  File "/opt/homebrew/Caskroom/miniconda/base/envs/memeatron/lib/python3.13/contextlib.py", line 148, in __exit__
    next(self.gen)
    ~~~~^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/envs/memeatron/lib/python3.13/site-packages/pip/_internal/cli/command_context.py", line 19, in main_context
    with self._main_context

## Local Inference on GPU
Model page: https://huggingface.co/nvidia/NVIDIA-Nemotron-Nano-9B-v2

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/nvidia/NVIDIA-Nemotron-Nano-9B-v2)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [3]:
'''
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("nvidia/NVIDIA-Nemotron-Nano-9B-v2")

# Load model (on GPU if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(
    "nvidia/NVIDIA-Nemotron-Nano-9B-v2",
    torch_dtype=torch.bfloat16,  # saves memory, works on GPU
    trust_remote_code=True,
    device_map="auto"
)

# Example text input
input_text = "Hello, how are you today?"

# Tokenize input
inputs = tokenizer(input_text, return_tensors="pt").to(device)

# Generate output (limited to 50 tokens)
outputs = model.generate(**inputs, max_new_tokens=50)

# Decode output
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Input:", input_text)
print("Output:", decoded_output)
'''

'\nfrom transformers import AutoTokenizer, AutoModelForCausalLM\nimport torch\n\n# Load tokenizer\ntokenizer = AutoTokenizer.from_pretrained("nvidia/NVIDIA-Nemotron-Nano-9B-v2")\n\n# Load model (on GPU if available)\ndevice = "cuda" if torch.cuda.is_available() else "cpu"\nmodel = AutoModelForCausalLM.from_pretrained(\n    "nvidia/NVIDIA-Nemotron-Nano-9B-v2",\n    torch_dtype=torch.bfloat16,  # saves memory, works on GPU\n    trust_remote_code=True,\n    device_map="auto"\n)\n\n# Example text input\ninput_text = "Hello, how are you today?"\n\n# Tokenize input\ninputs = tokenizer(input_text, return_tensors="pt").to(device)\n\n# Generate output (limited to 50 tokens)\noutputs = model.generate(**inputs, max_new_tokens=50)\n\n# Decode output\ndecoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)\n\nprint("Input:", input_text)\nprint("Output:", decoded_output)\n'

In [4]:
!pip install flask flask-ngrok transformers torch -q

from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [7]:
app = Flask(__name__)
run_with_ngrok(app)  # exposes public URL

# -------------------------------
# Load Nemotron-Nano-9B-v2
# -------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained("nvidia/NVIDIA-Nemotron-Nano-9B-v2")
model = AutoModelForCausalLM.from_pretrained(
    "nvidia/NVIDIA-Nemotron-Nano-9B-v2",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)

# -------------------------------
# Flask route for React frontend
# -------------------------------
@app.route("/api/respond", methods=["POST"])
def respond():
    data = request.json
    input_text = data.get("input", "")

    # Tokenize input
    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    # Generate output (50 tokens max)
    outputs = model.generate(**inputs, max_new_tokens=50)

    # Decode output
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return jsonify({
        "input": input_text,
        "output": decoded_output
    })

# -------------------------------
# Run Flask app
# -------------------------------
app.run()


ImportError: mamba-ssm is required by the Mamba model but cannot be imported

In [ ]:
# Clear the CUDA cache
torch.cuda.empty_cache()

# Delete variables that might be holding onto GPU memory
# You might need to add other variable names here if you have more objects on the GPU
if 'model' in locals():
    del model
if 'pipe' in locals():
    del pipe
if 'pipe_smaller' in locals():
    del pipe_smaller
if 'input_ids' in locals():
    del input_ids
if 'outputs' in locals():
    del outputs
if 'tokenizer' in locals():
    del tokenizer

# Run Python's garbage collector
import gc
gc.collect()

print("GPU memory hopefully cleared.")